# Using Sherlock out-of-the-box
This notebook shows how to predict a semantic type for a given table column.
The steps are basically:
- Download files for word embedding and paragraph vector feature extraction (downloads only once) and initialize feature extraction models.
- Extract features from table columns.
- Initialize Sherlock.
- Make a prediction for the feature representation of the column.

In [2]:
DIR_DATASET = '/ivi/inde/mmargaret/data_search_e_data_csv/'
DIR_LOG = '/ivi/inde/mmargaret/sherlock-project/log/'
DIR_OUTPUT = '/ivi/inde/mmargaret/sherlock-project/output/'

# DIR_DATASET = '/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/data/data_search_e_data_csv/'
# DIR_LOG = '/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/log/'
# DIR_OUTPUT = '/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/output_server/'

In [3]:
import logging
from datetime import datetime
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='{}{}'.format(DIR_LOG,datetime.now().strftime('%Y%m%d_%H%M_sherlock.log')), mode='a')
formatter = logging.Formatter('%(asctime)s - %(levelname)s : %(message)s', datefmt='%m/%d/%Y %I:%M')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

In [7]:
import numpy as np
import pandas as pd
import pyarrow as pa
import os
import sys
    
from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

ModuleNotFoundError: No module named 'pyarrow'

## Initialize feature extraction models

In [ ]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

### Get list of all dataset

In [ ]:
_ = os.listdir(DIR_DATASET)

In [ ]:
file_list = [id for id in _ if '.csv' in id]
logging.info('Number of Files: {}'.format(len(file_list)))
print('Number of Files: {}'.format(len(file_list)))

## Extract features

In [5]:
error_list=[]

def extractIDSemanticsWithColumnNames(filename):
    
    IDSemanticsColumns = {'data_filename':filename, 'colSemantics': [], 'colNames':[]}
    try:
        # read files
        with open(DIR_DATASET + filename, errors='ignore') as f:
            a_doc = pd.read_csv(f)
        a_doc = a_doc.astype(str) #only non-numeric object to str (sherlock required) = .select_dtypes(include=[object])
        data = pd.Series(a_doc.transpose().values.tolist(), name="values") #format it to list of values by columns

        # sherlock extract features
        extract_features("../temporary.csv",data)
        feature_vectors = pd.read_csv("../temporary.csv", dtype=np.float32)

        # sherlock init and predict with pre-trained model
        model = SherlockModel();
        model.initialize_model_from_json(with_weights=True, model_id="sherlock");
        predicted_labels = model.predict(feature_vectors, "sherlock")

        # return dictionary with id: id of the doc, list of the columns' semantics, list of the columns' names
        IDSemanticsColumns = {'data_filename':filename, 'colSemantics': list(predicted_labels), 'colNames':list(a_doc.columns)}
    
    except Exception as e:
        logging.error('Unable to extract: {}'.format(filename))
        print('Unable to extract: {}'.format(filename))
        
        print(e)
        logging.error(e, exc_info=True)
        
        global error_list
        error_list += [filename]
        
    return IDSemanticsColumns


In [27]:
# TEST function
logging.info('- TEST START -')

test_file = file_list[3]
print(test_file)
logging.info('filename: {}'.format(test_file))

test_extract = extractIDSemanticsWithColumnNames(test_file)
print (test_extract)
logging.info('extraction: {}'.format(test_extract))

print (error_list)
logging.info('error list: {}'.format(error_list))

logging.info('- TEST END -')


29c42fa751d277febbd914e65741513f63166637f9f18f824f685b4b2183dd21.text.csv


/Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: DtypeWarning: Columns (8,9,10,11,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  
Extracting Features:  13%|███                    | 2/15 [00:00<00:00, 15.26it/s]

Exporting 1588 column features


Extracting Features: 100%|██████████████████████| 15/15 [00:01<00:00, 14.40it/s]


Unable to extract: 29c42fa751d277febbd914e65741513f63166637f9f18f824f685b4b2183dd21.text.csv
'str' object has no attribute 'decode'
{'data_filename': '29c42fa751d277febbd914e65741513f63166637f9f18f824f685b4b2183dd21.text.csv', 'colSemantics': [], 'colNames': []}
['29c42fa751d277febbd914e65741513f63166637f9f18f824f685b4b2183dd21.text.csv']


### Get list of latest extracted semantics

In [6]:
_ = os.listdir(DIR_OUTPUT)
_

NameError: name 'os' is not defined

In [1]:
enrich_list = []

try:
    output_list = [DIR_OUTPUT + str(id) for id in _ if 'enriched_part_' in id]
    logging.info('Number of Output Files: {}'.format(len(output_list)))
    print('Number of Output Files: {}'.format(len(output_list)))

    latest_output = max(output_list, key=os.path.getctime)
    logging.info('Latest Output Filename: {}'.format(latest_output))
    print('Latest Output Filename: {}'.format(latest_output))

    output_df = pd.read_csv(latest_output)
    output_filenames = output_df['data_filename'].tolist()
    enrich_list = output_df.to_dict('records')
    
    logging.info('Number of Extracted Dataset: {}'.format(len(output_filenames)))
    print('Number of Extracted Dataset: {}'.format(len(output_filenames)))

except Exception as e:
    logging.error('Unable to retrieve latest output')
    print('Unable to retrieve latest output')
    
    logging.error(e, exc_info=True)
    print(e)
    pass

output_filenames[:5]

NameError: name 'logging' is not defined

### Start Extraction

In [ ]:
logging.info('- EXTRACT START -')

for i in range(0, len(file_list)):
    
    # so that it does not need to rerun existing output
    if (file_list[i] in output_filenames):
        logging.info('Existed: {} skipped'.format(file_list[i])
        print('Existed: {} skipped'.format(file_list[i])
        continue
        
    enrich_list += [extractIDSemanticsWithColumnNames(file_list[i])]
    if i%10==0:
        
        logging.info('i: {}'.format(i))
        sys.stdout.write('- i: {} -'.format(i))
        sys.stdout.write('\n')
        
        pd.DataFrame(enrich_list
                     , columns=['data_filename', 'colSemantics', 'colNames']).to_csv(DIR_OUTPUT +'enriched_part_' + str(i) +'.csv'
                     , index=False)
        
        pd.DataFrame(error_list
             , columns=['data_filename']).to_csv(DIR_OUTPUT + 'error_part_' + str(i) +'.csv'
             , index=False)
        
logging.info('- EXTRACT END -')
        

Extracting Features:  38%|█████████               | 3/8 [00:00<00:00, 20.54it/s]

Exporting 1588 column features


Extracting Features:  12%|██▊                    | 3/25 [00:00<00:02,  8.53it/s]

Exporting 1588 column features


Extracting Features: 100%|████████████████████████| 6/6 [00:00<00:00, 69.95it/s]


...

Extracting Features:   0%|                                | 0/8 [00:00<?, ?it/s]

Unable to extract: 2703ca716d09bc320efc521ca3ef614a129a339c7a35977d70a42bc51f1d8f5b.text.csv


Extracting Features:  25%|██████                  | 2/8 [00:00<00:01,  3.16it/s]

Exporting 1588 column features


Extracting Features: 100%|████████████████████████| 8/8 [00:00<00:00, 10.08it/s]


In [ ]:
pd.DataFrame(enrich_list
             , columns=['data_filename', 'colSemantics', 'colNames']).to_csv(DIR_OUTPUT +'enriched_all.csv'
             , index=False)


In [ ]:
pd.DataFrame(error_list
             , columns=['data_filename']).to_csv(DIR_OUTPUT + 'error_all.csv'
             , index=False)
